In [83]:
import json
import pandas as pd
from tqdm.auto import tqdm
from elasticsearch import Elasticsearch
from sentence_transformers import SentenceTransformer

In [84]:
with open('document_with_ids.json', 'rt') as f_in:
    documents = json.load(f_in)

In [85]:
model_name = 'multi-qa-MiniLM-L6-cos-v1'
model = SentenceTransformer(model_name)

### Indexing

In [86]:
for doc in tqdm(documents):
    question = doc['question']
    text = doc['text']
    qt = question + ' ' + text

    doc['question_vector'] = model.encode(question)
    doc['text_vector'] = model.encode(text)
    doc['question_text_vector'] = model.encode(qt)
    

  0%|          | 0/948 [00:00<?, ?it/s]

In [87]:
es_client = Elasticsearch("http://localhost:9200")

index_settings = {
    "settings": {
        "number_of_shards" : 1,
        "number_of_replicas" : 0
    },
    "mappings" : {
        "properties" : {
            "text" : {"type" : "text"},
            "section" : {"type" : "text"},
            "question" : {"type" : "text"},
            "course" : {"type" : "keyword"},
            "id" : {"type" : "keyword"},
            "question_vector" : {
                "type" : "dense_vector",
                "dims" : 384,
                "index" : True,
                "similarity": "cosine"
            },
            "text_vector" : {
                "type" : "dense_vector",
                "dims" : 384,
                "index" : True,
                "similarity": "cosine"
            },
            "question_text_vector" : {
                "type" : "dense_vector",
                "dims" : 384,
                "index" : True,
                "similarity": "cosine"
            },
        }
    }
}

index_name = "course_questions"
es_client.indices.delete(index = index_name, ignore_unavailable=True)
es_client.indices.create(index = index_name, body = index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course_questions'})

In [88]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

#### Retrieval stage

In [89]:
from langchain.embeddings import SentenceTransformerEmbeddings
from typing import Dict
from langchain_elasticsearch import ElasticsearchRetriever

In [90]:
es_url = "http://localhost:9200"

In [91]:
course = "data-engineering-zoomcamp"
query = "Can i still join the course?"
embeddings = SentenceTransformerEmbeddings(model_name = 'multi-qa-MiniLM-L6-cos-v1')

In [92]:
def hybrid_query(search_query: str) -> Dict:
    vector = embeddings.embed_query(search_query)  # same embeddings as for indexing
    return {
        "query" : {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": search_query,
                        "fields": ["question", "text", "section"],
                        "type": "best_fields",
                        "boost" :  0.9
                    }
                },
                "filter": {
                    "term": {
                        "course": course
                    }
                }
            }
        },
        "knn" : {
            "field" : "question_text_vector",
            "query_vector" : vector,
            "k" : 5,
            "num_candidates" : 10000,
            "boost" :  0.1,
            "filter" : {
                "term" : {
                    "course" : course
                        }
                    }
        },
        "size" : 5,
        #"rank" : {"rrf": {} },
    }


hybrid_retriever = ElasticsearchRetriever.from_es_params(
    index_name=index_name,
    body_func=hybrid_query,
    content_field="text",
    url=es_url,
)

In [93]:
hybrid_result = hybrid_retriever.invoke(query)

In [73]:
for result in hybrid_result:
    print(result.metadata['_source']['question'],result.metadata['_source']['course'], result.metadata['_score'])

Course - Can I still join the course after the start date? data-engineering-zoomcamp 19.887222
Course - Can I follow the course after it finishes? data-engineering-zoomcamp 11.013195
Course - What can I do before the course starts? data-engineering-zoomcamp 10.935482
Course - When will the course start? data-engineering-zoomcamp 10.732398
Course - Can I get support if I take the course in the self-paced mode? data-engineering-zoomcamp 10.2377405


## hybrid

In [94]:
df_ground_truth = pd.read_csv('ground_truth_data.csv')
ground_truth = df_ground_truth.to_dict(orient='records')

In [95]:
def hit_rate(relavance_total):
    cnt = 0
    for line in relavance_total:
        if True in line:
            cnt = cnt + 1
    return cnt / len(relavance_total)

In [96]:
def mrr_score(relavance_total):
    score = 0.0
    for line in relavance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                score = score + 1 / (rank + 1)
        
    return score / len(relavance_total)

In [103]:
#Reuse a single client/retriever instead of recreating it per request:

hybrid_retriever = ElasticsearchRetriever.from_es_params(
        index_name=index_name,
        body_func=hybrid_query,
        content_field="text",
        url=es_url,
    )

In [122]:
def elastic_search_hybrid(field, query, course):
    def hybrid_query(search_query: str) -> Dict:
        vector = embeddings.embed_query(search_query)  # same embeddings as for indexing
        return {
            "query" : {
                "bool": {
                    "must": {
                        "multi_match": {
                            "query": search_query,
                            "fields": ["question", "text", "section"],
                            "type": "best_fields",
                            "boost" :  0.4
                        }
                    },
                    "filter": {
                        "term": {
                            "course": course
                        }
                    }
                }
            },
            "knn" : {
                "field" : field,
                "query_vector" : vector,
                "k" : 5,
                "num_candidates" : 10000,
                "boost" :  0.6,
                "filter" : {
                    "term" : {
                        "course" : course
                            }
                        }
            },
            "size" : 5,
            "_source" : ["text", "section", "question", "course", "id"],
            #"rank" : {"rrf": {} },
        }
    #Reuse a single client/retriever instead of recreating it per request, move function outside
    
    hybrid_retriever = ElasticsearchRetriever.from_es_params(
        index_name=index_name,
        body_func=hybrid_query,
        content_field="text",
        url=es_url,
    )

    hybrid_result = hybrid_retriever.invoke(query)
    
    result_docs = []
    
    
    for hit in hybrid_result:
        result_docs.append(hit.metadata['_source'])
    
    return result_docs

In [123]:
ground_truth[0]

{'question': 'What is the starting date and time for the course?',
 'course': 'data-engineering-zoomcamp',
 'document': 'c02e79ef'}

In [124]:
question = ground_truth[0]['question']
course = ground_truth[0]['course']
elastic_search_hybrid('question_text_vector', question, course)

[{'section': 'General course-related questions',
  'question': 'Homework and Leaderboard - what is the system for points in the course management platform?',
  'course': 'data-engineering-zoomcamp',
  'id': '29865466'},
 {'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp',
  'id': '7842b56a'},
 {'section': 'General course-related questions',
  'question': 'Course - What are the prerequisites for this course?',
  'course': 'data-engineering-zoomcamp',
  'id': '1f6520ca'},
 {'section': 'Module 5: pyspark',
  'question': 'Homework - how to convert the time difference of two timestamps to hours',
  'course': 'data-engineering-zoomcamp',
  'id': 'f01df45b'},
 {'section': 'General course-related questions',
  'question': 'Homework - Are late submissions of homework allowed?',
  'course': 'data-engineering-zoomcamp',
  'id': 'be5bfee4'}]

In [125]:
def question_text_hybrid(q):
    question = q["question"]
    course = q["course"]

    return elastic_search_hybrid("question_text_vector", question, course)

In [126]:
from time import sleep

In [127]:
def evaluation_function(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['document']
        results = search_function(q)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)
        sleep(0.1)
    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr_score': mrr_score(relevance_total)
    }

In [128]:
evaluation_function(ground_truth, question_text_hybrid)

  0%|          | 0/4735 [00:00<?, ?it/s]

{'hit_rate': 0.9083421330517424, 'mrr_score': 0.8043083421330526}